In [ ]:
import pandas as pd
import numpy as np
import math
pd.options.mode.chained_assignment = None

**In this study we made a Employee Attrition Prediction model with ID3 Decision Tree Algorithm. We evaluated raw model's metrices then made a pruning on the tree and investigated to redundant features.**

In [ ]:
#with google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#read classification data with Pandas
df=pd.read_csv("/content/drive/MyDrive/Fall'22/deneme/ass-2/WA_Fn-UseC_-HR-Employee-Attrition.csv")

In [ ]:
#Drop employee number because all value is unique not effect by other person
df.drop(columns=["EmployeeNumber"],axis=1,inplace=True)
#shuffle data
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
#we searched categorical variable with labeled strings
from sklearn.preprocessing import LabelEncoder
other_categorical=list(set(df.columns) - set(df.describe().columns))
label_enc = LabelEncoder()
for i in other_categorical:
  labels=label_enc.fit_transform(df[i])
  print(list(zip(label_enc.classes_,range(len(label_enc.classes_)))))
  df[i] = labels
#here is the encoded label's pairs

[('Human Resources', 0), ('Life Sciences', 1), ('Marketing', 2), ('Medical', 3), ('Other', 4), ('Technical Degree', 5)]
[('Divorced', 0), ('Married', 1), ('Single', 2)]
[('Y', 0)]
[('No', 0), ('Yes', 1)]
[('Healthcare Representative', 0), ('Human Resources', 1), ('Laboratory Technician', 2), ('Manager', 3), ('Manufacturing Director', 4), ('Research Director', 5), ('Research Scientist', 6), ('Sales Executive', 7), ('Sales Representative', 8)]
[('No', 0), ('Yes', 1)]
[('Female', 0), ('Male', 1)]
[('Human Resources', 0), ('Research & Development', 1), ('Sales', 2)]
[('Non-Travel', 0), ('Travel_Frequently', 1), ('Travel_Rarely', 2)]


In [ ]:
#in dataset explanation the below columns was given 
categorical_given=["Education","EnvironmentSatisfaction","JobInvolvement","JobSatisfaction","PerformanceRating","RelationshipSatisfaction","WorkLifeBalance","StockOptionLevel","JobLevel"]
all_categorical=categorical_given+other_categorical

In [ ]:
y=df["Attrition"]
X=df.drop(columns=["Attrition"],axis=1)

# PART-1 DECISION TREE

In [ ]:
#create a node class for creating tree
class Node():
  def __init__(self):
    self.who=""
    self.childs=[]
    self.positive=0
    self.negative=0
    self.gain = 0

class decision_tree():
    def __init__(self,categorical):
      #we give categorical values in the begining because decision tree will do different calculations on categorical and continuous values
      self.categorical=categorical

    def fit(self,X_train,y_train):
      self.train=pd.concat([X_train, y_train], axis=1)
      #take note of the name of the attribute because we keep both X and y values in same dataframe
      self.target=y_train.name
      self.root=Node()
      self.rec(self.root,self.train)
      return self.root

    def rec(self,curr,df):

      #gain calculation, but different for categorical and continuoues, therefore it is separated as if else
      gains={}
      for i in df.columns:
        if(i!=self.target and i in self.categorical):
          gains[i]=self.gain(df,i)
        elif(i!=self.target and i not in self.categorical):
          threshold=self.best_split(df,i)
          gains[i]=self.continuous_gain(df,threshold,i)
      curr.gain = max(gains.values())
      #We find the maximum among the gains and assign the name of this maximum gain to max_value, so the max value is actually a string
      max_value = max(gains, key=gains.get)

      #We keep the true false numbers as positive and negative in the node because we choose the most common of the feature with these retained positives and negatives to avoid returning none if the features do not have any labels during estimation.
      curr.positive=df[df[self.target].eq(1)].shape[0]
      curr.negative=df[df[self.target].eq(0)].shape[0]


      #Since curr is equal to max gain, we give the name of the attribute to the node.
      curr.who=max_value
      #whether the max_value is categorical or not
      if(max_value in self.categorical):
        uniques=sorted(df[max_value].unique())
        for i in uniques:
          
          #if true or false comes 0 then it is equal to vice versa if both are false or true we need to get most common value
          true=df[(df[max_value]==i) & (df[self.target]==1)].shape[0]
          false=df[(df[max_value]==i) & (df[self.target]==0)].shape[0]

          #first three cases don't any function call for recusive, these are leaf nodes for the tree

          #if true and false equal to zero if any cases we can give most common attribute 
          if(true==0 and false==0):

            new_curr=Node()
            new_curr="No" if(curr.negative>curr.positive) else "Yes"
            new_curr.positive=true
            new_curr.negative=false
            curr.childs.append([i,new_curr])

          #when all is false then leaf node must be "Yes" in the other verb "1"
          elif(false==0):
            new_curr=Node()
            new_curr.who="Yes"
            new_curr.positive=true
            new_curr.negative=false
            curr.childs.append([i,new_curr])

          #when all is true then leaf node must be "No" in the other verb "0"
          elif(true==0):
            new_curr=Node()
            new_curr.who="No"
            new_curr.positive=true
            new_curr.negative=false
            curr.childs.append([i,new_curr])

          #if the tree have some true and false it must be grow with unused features
          else:
            df1=df[df[max_value].eq(i)]
            new_curr=Node()
            curr.childs.append([i,new_curr])
            #We droped the used attributes from the dataframe in categorical values ​​because they are already classified
            df1.drop([max_value],axis=1,inplace= True)
            self.rec(new_curr,df1)
      else:
        threshold=self.best_split(df,max_value)

        #left children is "<=threshold"

        true1=df[(df[max_value] <= threshold) & (df[self.target]==1)].shape[0]
        false1=df[(df[max_value] <= threshold) & (df[self.target]==0)].shape[0]

        #first three cases don't any function call for recusive, these are leaf nodes for the tree

        #if true and false equal to zero if any cases we can give most common attribute 
        if(true1==0 and false1==0):
          
          new_curr=Node()
          new_curr.who="No" if(curr.negative>curr.positive) else "Yes"
          new_curr.positive=true1
          new_curr.negative=false1
          curr.childs.append([str(threshold),"<=",new_curr])

        #when all is false then leaf node must be "Yes" in the other verb "1"
        elif(false1==0):
          new_curr=Node()
          new_curr.who="Yes"
          new_curr.positive=true1
          new_curr.negative=false1
          curr.childs.append([str(threshold),"<=",new_curr])

        #when all is true then leaf node must be "No" in the other verb "0"
        elif(true1==0):
          new_curr=Node()
          new_curr.who="No"
          new_curr.positive=true1
          new_curr.negative=false1
          curr.childs.append([str(threshold),"<=",new_curr])

        #if the tree have some true and false it must be grow with unused features
        else:
          df2=df[df[max_value]<=threshold]
          new_curr=Node()
          curr.childs.append([str(threshold),"<=",new_curr])
          self.rec(new_curr,df2)

        #right children is ">threshold"

        true2=df[(df[max_value] > threshold) & (df[self.target]==1)].shape[0]
        false2=df[(df[max_value] > threshold) & (df[self.target]==0)].shape[0]

        #first three cases don't any function call for recusive, these are leaf nodes for the tree

        #if true and false equal to zero if any cases we can give most common attribute 
        if(true2==0 and false2==0):
          new_curr=Node()
          new_curr.who="No" if(curr.negative>curr.positive) else "Yes"
          new_curr.positive=true2
          new_curr.negative=false2
          curr.childs.append([str(threshold),">",new_curr])

        #when all is true then leaf node must be "No" in the other verb "0"
        elif(false2==0):
          new_curr=Node()
          new_curr.who="Yes"
          new_curr.positive=true2
          new_curr.negative=false2
          curr.childs.append([str(threshold),">",new_curr])

        #when all is true then leaf node must be "No" in the other verb "0"
        elif(true2==0):
          new_curr=Node()
          new_curr.who="No"
          new_curr.positive=true2
          new_curr.negative=false2
          curr.childs.append([str(threshold),">",new_curr])

        #if the tree have some true and false it must be grow with unused features
        else:
          df3=df[df[max_value]>threshold]
          new_curr=Node()
          curr.childs.append([str(threshold),">",new_curr])
          self.rec(new_curr,df3)

      return curr

    #calculate entropy with number of 1(yes) and 0(no) numbers of Attrition
    def call_entropy(self,yes,no):
        return -((yes/(yes+no))*math.log2((yes/(yes+no))))-((no/(yes+no))*math.log2((no/(yes+no))))

    def gain(self,df,column):
        #calculate feature entropy with number of 1(yes) and 0(no) numbers of Attrition
        yes=df[df[self.target]==1].shape[0]
        no=df[df[self.target]==0].shape[0]
        E=self.call_entropy(yes,no) #main entropy

        #Calculate gain with for loop
        gain=E
        for i in df[column].unique():
          true=df[(df[column] == i) & (df[self.target]==1)].shape[0]
          false=df[(df[column] == i) & (df[self.target]==0)].shape[0] 
          
          #calculate entropy of any feature's label
          if(true!=0 and false!=0):
            e=self.call_entropy(true,false)
          else:
            e=0
          gain=gain-(((true+false)/(yes+no))*e) #subtract the partial(feature's labels) entropy values in main entropy for calculate gain

        return gain

    def best_split(self,df,column):
        df_uniq=sorted(df[column].unique())
        #possible thresholds - gain pairs created for get best threshold for continuous values
        possible_thresholds={}
        prev=0
        length_uniq=len(df_uniq)
        for i in range(length_uniq):
          if(i==0):
            prev=df_uniq[i]
          else:
            possible_thresholds[(prev+df_uniq[i])/2]=0
            prev=df_uniq[i]
        #if no any possible thresholds then we only have one label for threshold than threshold equal this label
        if(len(possible_thresholds)==0):
          possible_thresholds[df_uniq[0]]=0
        
        #calculate possible thresholds' gains then get maximum gain with its threshold
        for i in possible_thresholds:
          possible_thresholds[i]=self.continuous_gain(df,i,column)

        max_value = max(possible_thresholds, key=possible_thresholds.get)

        return max_value 

    def continuous_gain(self,df,threshold,column):
        #true1 and false1 correspond to less than or equal to threshold value
        true1=df[(df[column] <= threshold) & (df[self.target]==1)].shape[0]
        false1=df[(df[column] <= threshold) & (df[self.target]==0)].shape[0]  
        #true2 and false2 correspond to bigger than threshold value
        true2=df[(df[column] > threshold) & (df[self.target]==1)].shape[0]
        false2=df[(df[column] > threshold) & (df[self.target]==0)].shape[0]  
        
        #calculate main entropy for initalize gain
        E=self.call_entropy((true1+true2),(false1+false2))
        gain=E
        #calculate entropies of left and right side of threshold
        if(true1!=0 and false1!=0):
            e1=self.call_entropy(true1,false1)
        else:
            e1=0

        if(true2!=0 and false2!=0):
            e2=self.call_entropy(true2,false2)
        else:
            e2=0

        #with left and right side of entropy calculate gain
        gain=gain-(((true1+false1)/(true1+true2+false1+false2))*e1)

        gain=gain-(((true2+false2)/(true1+true2+false1+false2))*e2)

        return gain
    

    
    #for any none predict it returns most comman target value for prediction
    def is_none(self,node):
      if(node.positive>=node.negative):
        return 1
      else:
        return 0

    def predict(self,X_test):
        y_pred=[]
        #iterrate row for prediction part
        for i,row in X_test.iterrows():
            pred=self.make_predict(row,self.root)
            y_pred.append(pred)
        return y_pred

    def make_predict(self,row,tree):
      #base condition : if tree.who equal Yes or No then tree come to leaf node than return predict value
      if(tree.who=="Yes"):
        return 1
      elif(tree.who=="No"):
        return 0
      
      pred=None
      for i in tree.childs:
        if(tree.who in self.categorical and row[tree.who]==float(i[0])):
            pred=self.make_predict(row,i[1])
            #this condition for not get any predict so we calculate most comman target value for prediction
            if(pred==None):
              pred=self.is_none(tree)
            break
        if(tree.who not in self.categorical):
          if(i[1]==">" and row[tree.who]>float(i[0])):
            pred=self.make_predict(row,i[2])
            if(pred==None):
              pred=self.is_none(tree)
            break
          elif(i[1]=="<=" and row[tree.who]<=float(i[0])):
            pred=self.make_predict(row,i[2])
            if(pred==None):
              pred=self.is_none(tree)
            break
      return pred

    #Print tree with recursive function
    #when the tree in level n then it have n tab printing
    def print_tree(self,curr,number):
      if(number==0):
        print("Parent:",curr.who,"Positive:",curr.positive,"Negative:",curr.negative)
      
      if(curr.who in self.categorical):
        
          for i in curr.childs:
            print((number)*"\t",end="")
            if(i[1].who=="Yes" or i[1].who=="No"):
              print(" ",curr.who,"--",i[0],"-->",i[1].who,"Positive:",i[1].positive,"Negative:",i[1].negative)
              
            else:
              print(" ",curr.who,"--",i[0],"-->",i[1].who,"Positive:",i[1].positive,"Negative:",i[1].negative)

              self.print_tree(i[1],number+1)
      else:
          for i in curr.childs:
            print((number)*"\t",end="")
            if(i[2].who=="Yes" or i[2].who=="No"):
              print(" ",curr.who,"--",i[1]+i[0],"-->",i[2].who,"Positive:",i[2].positive,"Negative:",i[2].negative)

            else:
              print(" ",curr.who,"--",i[1]+i[0],"-->",i[2].who,"Positive:",i[2].positive,"Negative:",i[2].negative)
              self.print_tree(i[2],number+1)
    
    # pruning yapmak icin dallari buldugumuz recursive fonksiyon 
    def find_twigs(self, curr, twigs_list):
      if(len(curr.childs) != 1):                                  
        for child in reversed(curr.childs):                              # Yes No degerleri child listesinde sonda oldugu icin tersten dolastik
          if type(child[1]) == str:
            if child[2].who == "Yes" or child[2].who == "No":
              twigs_list.append([curr, curr.gain])
            else:
              twigs_list = self.find_twigs(child[2], twigs_list)
          else:
            if child[1].who == "Yes" or child[1].who == "No": 
              twigs_list.append([curr, curr.gain])
            else:
              twigs_list = self.find_twigs(child[1], twigs_list)


      return twigs_list                                                  # dallari return eder

    def pruning(self, twigs_list):

      global last_acc
      for twig in twigs_list:
        record = twig[0]                                                 # daha sonradan geri getirebilmek icin nodu kaydediyoruz
        if twig[0].positive > twig[0].negative: 
          twig[0].positive += twig[0].negative
          twig[0].negative = 0
          twig[0].childs[-1][-1].positive = twig[0].positive
          twig[0].childs[-1][-1].negative = 0
          twig[0].childs[-1][-1].who = "Yes"
          twig[0].childs = [twig[0].childs[-1]]
                  
        elif twig[0].positive < twig[0].negative:
          twig[0].negative += twig[0].positive
          twig[0].positive = 0
          twig[0].childs[-1][-1].negative = twig[0].negative
          twig[0].childs[-1][-1].positive = 0
          twig[0].childs[-1][-1].who = "No"
          twig[0].childs = [twig[0].childs[-1]]
        
        y_pred = self.predict(X_val)
        current_acc = accuracy_score(y_val, y_pred)
        if current_acc < last_acc:                                       # accuracy azalmissa islemi geri alir
          twig[0] = record
        else:                                                            # accuracy degismemisse veya artmissa 
          last_acc = current_acc
          new_twigs_list =  self.find_twigs(self.root, [])               # dal listesini yeniden olusturur
          new_twigs_list.sort(key=lambda x: x[1])                        
          self.pruning(new_twigs_list)                                   # yeniden pruning yapilir
          return last_acc
      return last_acc

## 5-FOLD CROSS VALIDATION RESULT AND COMMENTS

In [ ]:
clf=decision_tree(all_categorical)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
def cross_val(clf, X, y, cv):

        #split n-folds all data
        folds = np.array_split(pd.concat([X, y], axis=1), cv)
        metrics={}
        acc=-1
        best_root=None
        for i in range(cv):
            #iterate n times the train-test for each fold
            train = folds.copy()
            test = folds[i]
            del train[i]
            train = pd.concat(train, sort=False)

            X_train=train.iloc[:,:-1]
            y_train=train.iloc[:,-1]
            
            X_test=test.iloc[:,:-1]
            y_test=test.iloc[:,-1]
            
            #fit model to given classification algorithm
            root=clf.fit(X_train, y_train)
            #predict values
            y_pred=clf.predict(X_test)
            metric={}
            #keep the root with having most accuracy score
            if(accuracy_score(y_test, y_pred)>acc):
              best_root=root
              acc=accuracy_score(y_test, y_pred)
            metric['Accuracy']=accuracy_score(y_test, y_pred)
            metric['Precision']=precision_score(y_test, y_pred)
            metric['Recall']=recall_score(y_test, y_pred)
            metric['F1 Score']=f1_score(y_test, y_pred)
            metrics[i]=metric
            
        return (metrics,best_root)

      

In [ ]:
#get all metrices and best decision tree with its root
metrics,root=cross_val(clf, X, y, cv=5)

In [ ]:
df_metrices=pd.DataFrame(metrics).T

df_metrices.index.name="5-Fold Cross Validation"

df_metrices.set_index(df_metrices.index+1)

,Accuracy,Precision,Recall,F1 Score
5-Fold Cross Validation,,,,
1,0.758503,0.228070,0.325000,0.268041
2,0.809524,0.425926,0.479167,0.450980
3,0.782313,0.239130,0.275000,0.255814
4,0.734694,0.372549,0.292308,0.327586
5,0.802721,0.340909,0.340909,0.340909


In cross-validations the best accurcy is **0.809524** with **Fold-2** and the **F1 score is higher in Fold-2** so this is the best prediction for decision tree. 

We think that the tree accuracy can be more than these accuracy value because **we splited continuous values only 2 part.** With **more splits** the decision tree will have **more sensitivity.** 

Precision and Recall values ​​are low because test **target values ​​often have 0**. For this reason, **precision and recall states are affected** in case of even a  misclassification.



In [ ]:
#print to have best accuracy tree
clf.print_tree(root,0)

Parent: JobRole Positive: 189 Negative: 987
  JobRole -- 0 --> DistanceFromHome Positive: 8 Negative: 98
	  DistanceFromHome -- <=10.5 --> DailyRate Positive: 1 Negative: 68
		  DailyRate -- <=205.5 --> BusinessTravel Positive: 1 Negative: 4
			  BusinessTravel -- 0 --> No Positive: 0 Negative: 1
			  BusinessTravel -- 1 --> Yes Positive: 1 Negative: 0
			  BusinessTravel -- 2 --> No Positive: 0 Negative: 3
		  DailyRate -- >205.5 --> No Positive: 0 Negative: 64
	  DistanceFromHome -- >10.5 --> RelationshipSatisfaction Positive: 7 Negative: 30
		  RelationshipSatisfaction -- 1 --> StockOptionLevel Positive: 3 Negative: 3
			  StockOptionLevel -- 0 --> Yes Positive: 3 Negative: 0
			  StockOptionLevel -- 1 --> No Positive: 0 Negative: 3
		  RelationshipSatisfaction -- 2 --> DistanceFromHome Positive: 1 Negative: 6
			  DistanceFromHome -- <=28.0 --> No Positive: 0 Negative: 6
			  DistanceFromHome -- >28.0 --> Yes Positive: 1 Negative: 0
		  RelationshipSatisfaction -- 3 --> No Positive

The upper printed represent the best accuracy decision tree. If we give an example path;

```
JobRole -- 0 --> 
	 DistanceFromHome -- >10.5 -->
		  RelationshipSatisfaction -- 1 --> 
			    StockOptionLevel -- 1 --> No (Yes: 0 No: 3)
```

***COMPUTATION TIME:***

We goes on recursive with depth of tree it is assumed **N** and every recursion calling have for loop on columns and for loop on unique of the max gain column labels.

When we decide threshold for continuous value we iterate all unique values in columns for loop so the max columns length is **X** and max continuous unique values is **Y** then our computation time is ;

`O(N * X * Y) `

and duration of 5 fold crossvalidation was 6 minutes 37 seconds


#Comment for some misclassified samples

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [ ]:
root=clf.fit(X_train,y_train)

In [ ]:
y_pred=clf.predict(X_test)

In [ ]:
#get a misclassified example
index=[]
misclassified_size=0 #we look three of misclassified samples
count=0
for i,j in zip(y_pred,y_test):
  if(i!=j):
    print("misclassified example =",misclassified_size,"pred:",i,"test",j)
    index.append(y_test.index[count])
    misclassified_size+=1
  if(misclassified_size==3):
    break
  count+=1

misclassified example = 0 pred: 0 test 1
misclassified example = 1 pred: 1 test 0
misclassified example = 2 pred: 0 test 1


In [ ]:
#here is the tree for evaluating misclassified example
clf.print_tree(root,0)

Parent: JobRole Positive: 133 Negative: 749
  JobRole -- 0 --> EnvironmentSatisfaction Positive: 5 Negative: 77
	  EnvironmentSatisfaction -- 1 --> Education Positive: 4 Negative: 10
		  Education -- 1 --> No Positive: 0 Negative: 3
		  Education -- 2 --> Yes Positive: 1 Negative: 0
		  Education -- 3 --> Age Positive: 3 Negative: 2
			  Age -- <=45.5 --> Yes Positive: 3 Negative: 0
			  Age -- >45.5 --> No Positive: 0 Negative: 2
		  Education -- 4 --> No Positive: 0 Negative: 5
	  EnvironmentSatisfaction -- 2 --> DistanceFromHome Positive: 1 Negative: 10
		  DistanceFromHome -- <=26.5 --> No Positive: 0 Negative: 10
		  DistanceFromHome -- >26.5 --> Yes Positive: 1 Negative: 0
	  EnvironmentSatisfaction -- 3 --> No Positive: 0 Negative: 34
	  EnvironmentSatisfaction -- 4 --> No Positive: 0 Negative: 23
  JobRole -- 1 --> DistanceFromHome Positive: 5 Negative: 24
	  DistanceFromHome -- <=5.0 --> No Positive: 0 Negative: 16
	  DistanceFromHome -- >5.0 --> MaritalStatus Positive: 5 Nega

***First misclassified sample :***
-----------
*Predict: 0    True: 1*


The path is;


```
JobRole -- 2 --> 
  OverTime -- 0 --> 
    WorkLifeBalance -- 2 --> 
      MonthlyIncome -- 2367<=2650.0 --> 
        JobSatisfaction -- 4 --> No (Yes:0 - No:2)
```


In MonthlyIncome step there are 3 yes and 5 no so the tree is nearly balanced in this step the yes and no values hardly decrease so it affects the excellent classification. The pruning may solve this problem because it looks like overfitting has occurred.

*`The misclassified sample X values in the below.`*

In [ ]:
X_test.loc[index[0],:]

Age                            28
BusinessTravel                  2
DailyRate                    1404
Department                      1
DistanceFromHome               17
Education                       3
EducationField                  5
EmployeeCount                   1
EnvironmentSatisfaction         3
Gender                          1
HourlyRate                     32
JobInvolvement                  2
JobLevel                        1
JobRole                         2
JobSatisfaction                 4
MaritalStatus                   0
MonthlyIncome                2367
MonthlyRate                 18779
NumCompaniesWorked              5
Over18                          0
OverTime                        0
PercentSalaryHike              12
PerformanceRating               3
RelationshipSatisfaction        1
StandardHours                  80
StockOptionLevel                1
TotalWorkingYears               6
TrainingTimesLastYear           2
WorkLifeBalance                 2
YearsAtCompany

***Second misclassified sample :***
---------------
*Predict: 1    True: 0*


The path is;


```
JobRole -- 7 --> 
  StockOptionLevel -- 0 --> 
      EnvironmentSatisfaction -- 4 --> 
         EducationField -- 3 --> 
           DailyRate -- 1052>955.5 --> Yes (Yes:1 - No:0)
```


It is clear that the class in which it is classified, Yes, only 1 existed. This shows us that the overfit situation may have come into play. Also, if we had divided the value not by 2, but by 3 or 4, the probability of correct classification would be higher.


*`The misclassified sample X values in the below.`*

In [ ]:
X_test.loc[index[1],:]

Age                            45
BusinessTravel                  0
DailyRate                    1052
Department                      2
DistanceFromHome                6
Education                       3
EducationField                  3
EmployeeCount                   1
EnvironmentSatisfaction         4
Gender                          0
HourlyRate                     57
JobInvolvement                  2
JobLevel                        3
JobRole                         7
JobSatisfaction                 4
MaritalStatus                   2
MonthlyIncome                8865
MonthlyRate                 16840
NumCompaniesWorked              6
Over18                          0
OverTime                        0
PercentSalaryHike              12
PerformanceRating               3
RelationshipSatisfaction        4
StandardHours                  80
StockOptionLevel                0
TotalWorkingYears              23
TrainingTimesLastYear           2
WorkLifeBalance                 3
YearsAtCompany

***Third misclassified sample :***
-----------
*Predict: 0    True: 1*


The path is;


```
JobRole -- 7 --> 
  StockOptionLevel -- 0 --> 
      EnvironmentSatisfaction -- 4 --> 
         EducationField -- 2 --> No (Yes:0 - No:8)
```


Here, we have reached the class where 8 no values ​​appear. This may indicate that the number of classes with yes should increase. Because the number of values ​​that are yes in the data is less than the numbers.

*`The misclassified sample X values in the below.`*


In [ ]:
X_test.loc[index[2],:]

Age                            40
BusinessTravel                  2
DailyRate                     299
Department                      2
DistanceFromHome               25
Education                       4
EducationField                  2
EmployeeCount                   1
EnvironmentSatisfaction         4
Gender                          1
HourlyRate                     57
JobInvolvement                  2
JobLevel                        3
JobRole                         7
JobSatisfaction                 2
MaritalStatus                   2
MonthlyIncome                9094
MonthlyRate                 17235
NumCompaniesWorked              2
Over18                          0
OverTime                        1
PercentSalaryHike              12
PerformanceRating               3
RelationshipSatisfaction        3
StandardHours                  80
StockOptionLevel                0
TotalWorkingYears               9
TrainingTimesLastYear           2
WorkLifeBalance                 3
YearsAtCompany

# Part 2: Pruning Decision Tree

In [ ]:
# Split Test, Validation, Train set for Pruning
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# %60 -> Train
# %20 -> Validation
# %20 -> Test

X_temp, X_val, y_temp,  y_val = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.25, random_state=0)      

root=clf.fit(X_train, y_train)

twigs_list = clf.find_twigs(root, [])
twigs_list.sort(key=lambda x: x[1])
y_pred=clf.predict(X_val)
last_acc = accuracy_score(y_val, y_pred)
before_pru_acc = last_acc

In [ ]:
clf.print_tree(root,0)

Parent: JobRole Positive: 147 Negative: 735
  JobRole -- 0 --> PercentSalaryHike Positive: 3 Negative: 69
	  PercentSalaryHike -- <=12.5 --> DistanceFromHome Positive: 3 Negative: 15
		  DistanceFromHome -- <=13.0 --> No Positive: 0 Negative: 13
		  DistanceFromHome -- >13.0 --> DistanceFromHome Positive: 3 Negative: 2
			  DistanceFromHome -- <=24.5 --> Yes Positive: 3 Negative: 0
			  DistanceFromHome -- >24.5 --> No Positive: 0 Negative: 2
	  PercentSalaryHike -- >12.5 --> No Positive: 0 Negative: 54
  JobRole -- 1 --> JobSatisfaction Positive: 9 Negative: 24
	  JobSatisfaction -- 1 --> DailyRate Positive: 4 Negative: 1
		  DailyRate -- <=341.0 --> No Positive: 0 Negative: 1
		  DailyRate -- >341.0 --> Yes Positive: 4 Negative: 0
	  JobSatisfaction -- 2 --> Age Positive: 2 Negative: 12
		  Age -- <=28.0 --> Age Positive: 2 Negative: 1
			  Age -- <=25.5 --> No Positive: 0 Negative: 1
			  Age -- >25.5 --> Yes Positive: 2 Negative: 0
		  Age -- >28.0 --> No Positive: 0 Negative: 11
	

In [ ]:
pruning_acc = clf.pruning(twigs_list)
print("Accuracy of Tree Before Pruning: ", before_pru_acc,"\nAccuracy of Tree After Pruning: ", pruning_acc)

Accuracy of Tree Before Pruning:  0.7959183673469388 
Accuracy of Tree After Pruning:  0.8673469387755102


# As a Conclusion
A large tree can cause overfitting. Pruning should reduce the size of a decision tree without reducing predictive accuracy. Pruning the tree prevents overfitting, increasing accuracy.

In [ ]:
clf.print_tree(root,0)

Parent: JobRole Positive: 147 Negative: 735
  JobRole -- 0 --> PercentSalaryHike Positive: 0 Negative: 72
	  PercentSalaryHike -- >12.5 --> No Positive: 0 Negative: 72
  JobRole -- 1 --> JobSatisfaction Positive: 9 Negative: 24
	  JobSatisfaction -- 1 --> DailyRate Positive: 5 Negative: 0
		  DailyRate -- >341.0 --> Yes Positive: 5 Negative: 0
	  JobSatisfaction -- 2 --> Age Positive: 0 Negative: 14
		  Age -- >28.0 --> No Positive: 0 Negative: 14
	  JobSatisfaction -- 3 --> DistanceFromHome Positive: 0 Negative: 8
		  DistanceFromHome -- >8.5 --> No Positive: 0 Negative: 8
	  JobSatisfaction -- 4 --> Age Positive: 0 Negative: 6
		  Age -- >23.5 --> No Positive: 0 Negative: 6
  JobRole -- 2 --> WorkLifeBalance Positive: 35 Negative: 118
	  WorkLifeBalance -- 1 --> JobSatisfaction Positive: 10 Negative: 0
		  JobSatisfaction -- 4 --> Yes Positive: 10 Negative: 0
	  WorkLifeBalance -- 2 --> DistanceFromHome Positive: 10 Negative: 25
		  DistanceFromHome -- <=7.5 --> YearsAtCompany Positi

# When the 28 Twigs shown below were removed, the accuracy increased.



```
# JobRole -- 0 --> PercentSalaryHike Positive: 3 Negative: 69
	  PercentSalaryHike -- <=12.5 --> DistanceFromHome Positive: 3 Negative: 15
		  DistanceFromHome -- <=13.0 --> No Positive: 0 Negative: 13
		  DistanceFromHome -- >13.0 --> DistanceFromHome Positive: 3 Negative: 2
		    DistanceFromHome -- <=24.5 --> Yes Positive: 3 Negative: 0
		    DistanceFromHome -- >24.5 --> No Positive: 0 Negative: 2
```


After Pruning :  JobRole -- 0 --> PercentSalaryHike Positive: 0 Negative: 72



```
# JobSatisfaction -- 1 --> DailyRate Positive: 4 Negative: 1
		  DailyRate -- <=341.0 --> No Positive: 0 Negative: 1
		  DailyRate -- >341.0 --> Yes Positive: 4 Negative: 0
```


After Pruning :  JobSatisfaction -- 1 --> DailyRate Positive: 5 Negative: 0



```
# JobSatisfaction -- 2 --> Age Positive: 2 Negative: 12
		  Age -- <=28.0 --> Age Positive: 2 Negative: 1
			  Age -- <=25.5 --> No Positive: 0 Negative: 1
			  Age -- >25.5 --> Yes Positive: 2 Negative: 0
		  Age -- >28.0 --> No Positive: 0 Negative: 11
```


After Pruning : JobSatisfaction -- 2 --> Age Positive: 0 Negative: 14



```
# JobSatisfaction -- 3 --> DistanceFromHome Positive: 2 Negative: 6
		  DistanceFromHome -- <=8.5 --> No Positive: 0 Negative: 5
		  DistanceFromHome -- >8.5 --> Age Positive: 2 Negative: 1
			  Age -- <=36.0 --> Yes Positive: 2 Negative: 0
			  Age -- >36.0 --> No Positive: 0 Negative: 1
```


After Pruning :  JobSatisfaction -- 3 --> DistanceFromHome Positive: 0 Negative: 8



```
# WorkLifeBalance -- 1 --> JobSatisfaction Positive: 7 Negative: 3
		  JobSatisfaction -- 1 --> Yes Positive: 3 Negative: 0
		  JobSatisfaction -- 2 --> No Positive: 0 Negative: 3
		  JobSatisfaction -- 3 --> Yes Positive: 3 Negative: 0
		  JobSatisfaction -- 4 --> Yes Positive: 1 Negative: 0
```



After Pruning : WorkLifeBalance -- 1 --> JobSatisfaction Positive: 10 Negative: 0



```
# DistanceFromHome -- <=7.5 --> YearsAtCompany Positive: 3 Negative: 20
			  YearsAtCompany -- <=2.5 --> HourlyRate Positive: 3 Negative: 8
				  HourlyRate -- <=58.0 --> Age Positive: 3 Negative: 1
					  Age -- <=26.5 --> No Positive: 0 Negative: 1
					  Age -- >26.5 --> Yes Positive: 3 Negative: 0
				  HourlyRate -- >58.0 --> No Positive: 0 Negative: 7
			  YearsAtCompany -- >2.5 --> No Positive: 0 Negative: 12
```



After Pruning : DistanceFromHome -- <=7.5 --> YearsAtCompany Positive: 0 Negative: 23



```
# DistanceFromHome -- >7.5 --> MonthlyIncome Positive: 7 Negative: 5
			  MonthlyIncome -- <=2772.5 --> Yes Positive: 5 Negative: 0
			  MonthlyIncome -- >2772.5 --> DailyRate Positive: 2 Negative: 5
				  DailyRate -- <=691.0 --> Yes Positive: 2 Negative: 0
				  DailyRate -- >691.0 --> No Positive: 0 Negative: 5
```



After Pruning : DistanceFromHome -- >7.5 --> MonthlyIncome Positive: 12 Negative: 0



```
# Age -- <=23.5 --> JobSatisfaction Positive: 5 Negative: 3
			  JobSatisfaction -- 1 --> Yes Positive: 1 Negative: 0
			  JobSatisfaction -- 2 --> No Positive: 0 Negative: 1
			  JobSatisfaction -- 3 --> Yes Positive: 3 Negative: 0
			  JobSatisfaction -- 4 --> Age Positive: 1 Negative: 2
				  Age -- <=21.0 --> Yes Positive: 1 Negative: 0
				  Age -- >21.0 --> No Positive: 0 Negative: 2
```



After Pruning : Age -- <=23.5 --> JobSatisfaction Positive: 8 Negative: 0



```
# EnvironmentSatisfaction -- 1 --> JobInvolvement Positive: 6 Negative: 12
				  JobInvolvement -- 1 --> Yes Positive: 2 Negative: 0
				  JobInvolvement -- 2 --> DailyRate Positive: 2 Negative: 3
					  DailyRate -- <=864.5 --> No Positive: 0 Negative: 3
					  DailyRate -- >864.5 --> Yes Positive: 2 Negative: 0
				  JobInvolvement -- 3 --> Age Positive: 2 Negative: 6
					  Age -- <=26.5 --> Yes Positive: 2 Negative: 0
					  Age -- >26.5 --> No Positive: 0 Negative: 6
				  JobInvolvement -- 4 --> No Positive: 0 Negative: 3
```



After Pruning : EnvironmentSatisfaction -- 1 --> JobInvolvement Positive: 0 Negative: 18



```
# EnvironmentSatisfaction -- 2 --> TotalWorkingYears Positive: 1 Negative: 11
				  TotalWorkingYears -- <=2.0 --> Yes Positive: 1 Negative: 0
				  TotalWorkingYears -- >2.0 --> No Positive: 0 Negative: 11
```



After Pruning : EnvironmentSatisfaction -- 2 --> TotalWorkingYears Positive: 0 Negative: 12



```
# EnvironmentSatisfaction -- 3 --> DailyRate Positive: 2 Negative: 20
				  DailyRate -- <=107.0 --> Yes Positive: 1 Negative: 0
				  DailyRate -- >107.0 --> TotalWorkingYears Positive: 1 Negative: 20
					  TotalWorkingYears -- <=17.5 --> No Positive: 0 Negative: 20
					  TotalWorkingYears -- >17.5 --> Yes Positive: 1 Negative: 0
```



After Pruning : EnvironmentSatisfaction -- 3 --> DailyRate Positive: 0 Negative: 22



```
# EnvironmentSatisfaction -- 4 --> MonthlyIncome Positive: 1 Negative: 31
				  MonthlyIncome -- <=1547.5 --> Age Positive: 1 Negative: 1
					  Age -- <=31.5 --> No Positive: 0 Negative: 1
					  Age -- >31.5 --> Yes Positive: 1 Negative: 0
				  MonthlyIncome -- >1547.5 --> No Positive: 0 Negative: 30
```



After Pruning : EnvironmentSatisfaction -- 4 --> MonthlyIncome Positive: 0 Negative: 32



```
# JobRole -- 3 --> MonthlyRate Positive: 5 Negative: 65
	  MonthlyRate -- <=24537.5 --> OverTime Positive: 3 Negative: 65
		  OverTime -- 0 --> No Positive: 0 Negative: 51
		  OverTime -- 1 --> RelationshipSatisfaction Positive: 3 Negative: 14
			  RelationshipSatisfaction -- 1 --> Age Positive: 1 Negative: 1
				  Age -- <=44.5 --> No Positive: 0 Negative: 1
				  Age -- >44.5 --> Yes Positive: 1 Negative: 0
			  RelationshipSatisfaction -- 2 --> No Positive: 0 Negative: 2
			  RelationshipSatisfaction -- 3 --> No Positive: 0 Negative: 9
			  RelationshipSatisfaction -- 4 --> Department Positive: 2 Negative: 2
				  Department -- 1 --> Yes Positive: 2 Negative: 0
				  Department -- 2 --> No Positive: 0 Negative: 2
	  MonthlyRate -- >24537.5 --> Yes Positive: 2 Negative: 0
```



After Pruning : JobRole -- 3 --> MonthlyRate Positive: 0 Negative: 70



```
# JobRole -- 4 --> DailyRate Positive: 5 Negative: 76
	  DailyRate -- <=639.0 --> MonthlyIncome Positive: 5 Negative: 25
		  MonthlyIncome -- <=4237.5 --> Yes Positive: 2 Negative: 0
		  MonthlyIncome -- >4237.5 --> Education Positive: 3 Negative: 25
			  Education -- 1 --> No Positive: 0 Negative: 2
			  Education -- 2 --> YearsAtCompany Positive: 1 Negative: 8
				  YearsAtCompany -- <=1.5 --> Yes Positive: 1 Negative: 0
				  YearsAtCompany -- >1.5 --> No Positive: 0 Negative: 8
			  Education -- 3 --> No Positive: 0 Negative: 9
			  Education -- 4 --> Age Positive: 1 Negative: 6
				  Age -- <=31.5 --> Yes Positive: 1 Negative: 0
				  Age -- >31.5 --> No Positive: 0 Negative: 6
			  Education -- 5 --> Yes Positive: 1 Negative: 0
	  DailyRate -- >639.0 --> No Positive: 0 Negative: 51

```

After Pruning : JobRole -- 4 --> DailyRate Positive: 0 Negative: 81



```
# JobRole -- 5 --> YearsInCurrentRole Positive: 2 Negative: 44
	  YearsInCurrentRole -- <=14.5 --> No Positive: 0 Negative: 42
	  YearsInCurrentRole -- >14.5 --> HourlyRate Positive: 2 Negative: 2
		  HourlyRate -- <=50.0 --> Yes Positive: 2 Negative: 0
		  HourlyRate -- >50.0 --> No Positive: 0 Negative: 2

```

After Pruning : JobRole -- 5 --> YearsInCurrentRole Positive: 0 Negative: 46



```
# RelationshipSatisfaction -- 1 --> EnvironmentSatisfaction Positive: 5 Negative: 15
			  EnvironmentSatisfaction -- 1 --> BusinessTravel Positive: 1 Negative: 5
				  BusinessTravel -- 0 --> Yes Positive: 1 Negative: 0
				  BusinessTravel -- 2 --> No Positive: 0 Negative: 5
			  EnvironmentSatisfaction -- 2 --> No Positive: 0 Negative: 5
			  EnvironmentSatisfaction -- 3 --> DistanceFromHome Positive: 1 Negative: 5
				  DistanceFromHome -- <=21.0 --> No Positive: 0 Negative: 5
				  DistanceFromHome -- >21.0 --> Yes Positive: 1 Negative: 0
			  EnvironmentSatisfaction -- 4 --> Yes Positive: 3 Negative: 0

```

After Pruning : RelationshipSatisfaction -- 1 --> EnvironmentSatisfaction Positive: 0 Negative: 20
	  
			 



```
#  RelationshipSatisfaction -- 2 --> Education Positive: 1 Negative: 20
			  Education -- 2 --> No Positive: 0 Negative: 1
			  Education -- 3 --> No Positive: 0 Negative: 7
			  Education -- 4 --> No Positive: 0 Negative: 12
			  Education -- 5 --> Yes Positive: 1 Negative: 0

```

After Pruning : RelationshipSatisfaction -- 2 --> Education Positive: 0 Negative: 21



```
# RelationshipSatisfaction -- 3 --> HourlyRate Positive: 1 Negative: 30
			  HourlyRate -- <=98.5 --> No Positive: 0 Negative: 30
			  HourlyRate -- >98.5 --> Yes Positive: 1 Negative: 0

```

After Pruning : RelationshipSatisfaction -- 3 --> HourlyRate Positive: 0 Negative: 31



```
# RelationshipSatisfaction -- 4 --> NumCompaniesWorked Positive: 1 Negative: 40
			  NumCompaniesWorked -- <=5.5 --> No Positive: 0 Negative: 36
			  NumCompaniesWorked -- >5.5 --> HourlyRate Positive: 1 Negative: 4
				  HourlyRate -- <=50.0 --> Yes Positive: 1 Negative: 0
				  HourlyRate -- >50.0 --> No Positive: 0 Negative: 4

```

After Pruning : RelationshipSatisfaction -- 4 --> NumCompaniesWorked Positive: 0 Negative: 41



```
#  MonthlyIncome -- <=2475.0 --> EnvironmentSatisfaction Positive: 11 Negative: 4
			  EnvironmentSatisfaction -- 1 --> Yes Positive: 2 Negative: 0
			  EnvironmentSatisfaction -- 2 --> No Positive: 0 Negative: 2
			  EnvironmentSatisfaction -- 3 --> Yes Positive: 3 Negative: 0
			  EnvironmentSatisfaction -- 4 --> Age Positive: 6 Negative: 2
				  Age -- <=36.0 --> Yes Positive: 6 Negative: 0
				  Age -- >36.0 --> No Positive: 0 Negative: 2
		  

```

After Pruning :  MonthlyIncome -- <=2475.0 --> EnvironmentSatisfaction Positive: 15 Negative: 0
			 



```
# MonthlyIncome -- >2475.0 --> NumCompaniesWorked Positive: 9 Negative: 33
			  NumCompaniesWorked -- <=8.5 --> MonthlyIncome Positive: 7 Negative: 33
				  MonthlyIncome -- <=3924.0 --> EnvironmentSatisfaction Positive: 7 Negative: 18
					  EnvironmentSatisfaction -- 1 --> Age Positive: 1 Negative: 1
						  Age -- <=26.0 --> No Positive: 0 Negative: 1
						  Age -- >26.0 --> Yes Positive: 1 Negative: 0
					  EnvironmentSatisfaction -- 2 --> JobInvolvement Positive: 1 Negative: 7
						  JobInvolvement -- 1 --> Yes Positive: 1 Negative: 0
						  JobInvolvement -- 2 --> No Positive: 0 Negative: 3
						  JobInvolvement -- 3 --> No Positive: 0 Negative: 2
						  JobInvolvement -- 4 --> No Positive: 0 Negative: 2
					  EnvironmentSatisfaction -- 3 --> Age Positive: 1 Negative: 8
						  Age -- <=38.5 --> No Positive: 0 Negative: 8
						  Age -- >38.5 --> Yes Positive: 1 Negative: 0
					  EnvironmentSatisfaction -- 4 --> BusinessTravel Positive: 4 Negative: 2
						  BusinessTravel -- 1 --> No Positive: 0 Negative: 2
						  BusinessTravel -- 2 --> Yes Positive: 4 Negative: 0
				  MonthlyIncome -- >3924.0 --> No Positive: 0 Negative: 15
			  NumCompaniesWorked -- >8.5 --> Yes Positive: 2 Negative: 0

```

After Pruning : MonthlyIncome -- >2475.0 --> NumCompaniesWorked Positive: 0 Negative: 42



```
# StockOptionLevel -- 0 --> WorkLifeBalance Positive: 27 Negative: 62
		  WorkLifeBalance -- 1 --> Yes Positive: 4 Negative: 0
		  WorkLifeBalance -- 2 --> NumCompaniesWorked Positive: 13 Negative: 9
			  NumCompaniesWorked -- <=3.5 --> JobSatisfaction Positive: 3 Negative: 8
				  JobSatisfaction -- 1 --> Yes Positive: 2 Negative: 0
				  JobSatisfaction -- 2 --> No Positive: 0 Negative: 1
				  JobSatisfaction -- 3 --> No Positive: 0 Negative: 3
				  JobSatisfaction -- 4 --> DailyRate Positive: 1 Negative: 4
					  DailyRate -- <=991.0 --> No Positive: 0 Negative: 4
					  DailyRate -- >991.0 --> Yes Positive: 1 Negative: 0
			  NumCompaniesWorked -- >3.5 --> Age Positive: 10 Negative: 1
				  Age -- <=57.0 --> Yes Positive: 10 Negative: 0
				  Age -- >57.0 --> No Positive: 0 Negative: 1
		  WorkLifeBalance -- 3 --> MonthlyIncome Positive: 10 Negative: 41
			  MonthlyIncome -- <=9316.5 --> TotalWorkingYears Positive: 6 Negative: 39
				  TotalWorkingYears -- <=8.5 --> EnvironmentSatisfaction Positive: 6 Negative: 20
					  EnvironmentSatisfaction -- 1 --> RelationshipSatisfaction Positive: 3 Negative: 2
						  RelationshipSatisfaction -- 1 --> Yes Positive: 2 Negative: 0
						  RelationshipSatisfaction -- 3 --> Yes Positive: 1 Negative: 0
						  RelationshipSatisfaction -- 4 --> No Positive: 0 Negative: 2
					  EnvironmentSatisfaction -- 2 --> DailyRate Positive: 2 Negative: 4
						  DailyRate -- <=739.5 --> Yes Positive: 2 Negative: 0
						  DailyRate -- >739.5 --> No Positive: 0 Negative: 4
					  EnvironmentSatisfaction -- 3 --> No Positive: 0 Negative: 7
					  EnvironmentSatisfaction -- 4 --> Age Positive: 1 Negative: 7
						  Age -- <=26.5 --> Yes Positive: 1 Negative: 0
						  Age -- >26.5 --> No Positive: 0 Negative: 7
				  TotalWorkingYears -- >8.5 --> No Positive: 0 Negative: 19
			  MonthlyIncome -- >9316.5 --> EnvironmentSatisfaction Positive: 4 Negative: 2
				  EnvironmentSatisfaction -- 1 --> Yes Positive: 1 Negative: 0
				  EnvironmentSatisfaction -- 2 --> No Positive: 0 Negative: 2
				  EnvironmentSatisfaction -- 3 --> Yes Positive: 1 Negative: 0
				  EnvironmentSatisfaction -- 4 --> Yes Positive: 2 Negative: 0
		  WorkLifeBalance -- 4 --> No Positive: 0 Negative: 12
	  

```

After Pruning : StockOptionLevel -- 0 --> WorkLifeBalance Positive: 0 Negative: 89
	  



```
# StockOptionLevel -- 1 --> Education Positive: 4 Negative: 66
		  Education -- 1 --> Age Positive: 2 Negative: 1
			  Age -- <=37.5 --> Yes Positive: 2 Negative: 0
			  Age -- >37.5 --> No Positive: 0 Negative: 1
		  Education -- 2 --> No Positive: 0 Negative: 18
		  Education -- 3 --> JobInvolvement Positive: 2 Negative: 27
			  JobInvolvement -- 1 --> Yes Positive: 1 Negative: 0
			  JobInvolvement -- 2 --> DistanceFromHome Positive: 1 Negative: 10
				  DistanceFromHome -- <=1.5 --> Yes Positive: 1 Negative: 0
				  DistanceFromHome -- >1.5 --> No Positive: 0 Negative: 10
			  JobInvolvement -- 3 --> No Positive: 0 Negative: 14
			  JobInvolvement -- 4 --> No Positive: 0 Negative: 3
		  Education -- 4 --> No Positive: 0 Negative: 16
		  Education -- 5 --> No Positive: 0 Negative: 4
	  

```

After Pruning : StockOptionLevel -- 1 --> Education Positive: 0 Negative: 70



```
# StockOptionLevel -- 2 --> DistanceFromHome Positive: 3 Negative: 27
		  DistanceFromHome -- <=10.5 --> No Positive: 0 Negative: 20
		  DistanceFromHome -- >10.5 --> JobSatisfaction Positive: 3 Negative: 7
			  JobSatisfaction -- 1 --> Yes Positive: 1 Negative: 0
			  JobSatisfaction -- 2 --> No Positive: 0 Negative: 3
			  JobSatisfaction -- 3 --> No Positive: 0 Negative: 3
			  JobSatisfaction -- 4 --> DailyRate Positive: 2 Negative: 1
				  DailyRate -- <=1341.0 --> Yes Positive: 2 Negative: 0
				  DailyRate -- >1341.0 --> No Positive: 0 Negative: 1
	  

```

After Pruning : StockOptionLevel -- 2 --> DistanceFromHome Positive: 0 Negative: 30



```
# StockOptionLevel -- 3 --> DistanceFromHome Positive: 3 Negative: 10
		  DistanceFromHome -- <=9.5 --> No Positive: 0 Negative: 8
		  DistanceFromHome -- >9.5 --> Education Positive: 3 Negative: 2
			  Education -- 3 --> Yes Positive: 2 Negative: 0
			  Education -- 4 --> No Positive: 0 Negative: 2
			  Education -- 5 --> Yes Positive: 1 Negative: 0

```

After Pruning : StockOptionLevel -- 3 --> DistanceFromHome Positive: 0 Negative: 13



```
# WorkLifeBalance -- 4 --> EnvironmentSatisfaction Positive: 3 Negative: 13
		  EnvironmentSatisfaction -- 1 --> Yes Positive: 1 Negative: 0
		  EnvironmentSatisfaction -- 2 --> No Positive: 0 Negative: 6
		  EnvironmentSatisfaction -- 3 --> Age Positive: 1 Negative: 3
			  Age -- <=35.5 --> Yes Positive: 1 Negative: 0
			  Age -- >35.5 --> No Positive: 0 Negative: 3
		  EnvironmentSatisfaction -- 4 --> DailyRate Positive: 1 Negative: 4
			  DailyRate -- <=1171.5 --> No Positive: 0 Negative: 4
			  DailyRate -- >1171.5 --> Yes Positive: 1 Negative: 0
```



After Pruning : WorkLifeBalance -- 4 --> EnvironmentSatisfaction Positive: 0 Negative: 16



```
# WorkLifeBalance -- 1 --> JobSatisfaction Positive: 7 Negative: 3
		  JobSatisfaction -- 1 --> Yes Positive: 3 Negative: 0
		  JobSatisfaction -- 2 --> No Positive: 0 Negative: 3
		  JobSatisfaction -- 3 --> Yes Positive: 3 Negative: 0
		  JobSatisfaction -- 4 --> Yes Positive: 1 Negative: 0
```



After Pruning : WorkLifeBalance -- 1 --> JobSatisfaction Positive: 10 Negative: 0



```
# WorkLifeBalance -- 1 --> JobSatisfaction Positive: 7 Negative: 3
		  JobSatisfaction -- 1 --> Yes Positive: 3 Negative: 0
		  JobSatisfaction -- 2 --> No Positive: 0 Negative: 3
		  JobSatisfaction -- 3 --> Yes Positive: 3 Negative: 0
		  JobSatisfaction -- 4 --> Yes Positive: 1 Negative: 0
```



After Pruning : WorkLifeBalance -- 1 --> JobSatisfaction Positive: 10 Negative: 0